In [1]:
!pip install pyspark findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 23.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=85c4c9b8b6bd4b8f4523e08cee88cd0ac72310836c0a46d366fbe1fad5de157f
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Создание DataFrame**

Из RDD

In [4]:
def cleaning(row):
    row = row.split('\t')[:3]
    row = [float(val) for val in row]
    return row

In [5]:
ratings = sc.textFile('user_ratedmovies.dat')

first_row = ratings.first()
ratings = ratings.filter(lambda row: row != first_row)\
                 .map(cleaning)

Py4JJavaError: ignored

In [15]:
columns = first_row.split('\t')[:3]

In [16]:
columns

['userID', 'movieID', 'rating']

In [17]:
ratings.take(5)

[[75.0, 3.0, 1.0],
 [75.0, 32.0, 4.5],
 [75.0, 110.0, 4.0],
 [75.0, 160.0, 2.0],
 [75.0, 163.0, 4.0]]

In [18]:
df_rdd = spark.createDataFrame(ratings, columns)

In [19]:
df_rdd

DataFrame[userID: double, movieID: double, rating: double]

Можно еще вот так:

In [20]:
df_rdd = ratings.toDF(columns)

In [21]:
df_rdd

DataFrame[userID: double, movieID: double, rating: double]

Так, а если не хочу вот эти приседания с RDD, а хочу сразу из файла?

In [22]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .load("user_ratedmovies.dat")

In [23]:
df

DataFrame[userID: string, movieID: string, rating: string, date_day: string, date_month: string, date_year: string, date_hour: string, date_minute: string, date_second: string]

Все в string, так не пойдет, давайте автоматически определим тип данных

In [24]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true', 'inferSchema': 'true'})\
          .load("user_ratedmovies.dat")

In [25]:
df

DataFrame[userID: int, movieID: int, rating: double, date_day: int, date_month: int, date_year: int, date_hour: int, date_minute: int, date_second: int]

In [26]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

А можно заранее сказать, какой тип данных я ожидаю?

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, FloatType

In [28]:
schema = StructType([ \
    StructField("userID",IntegerType(),True), \
    StructField("movieID",IntegerType(),True), \
    StructField("rating",DoubleType(),True), \
    StructField("date_day", StringType(), True), \
    StructField("date_month", StringType(), True), \
    StructField("date_year", IntegerType(), True), \
    StructField("date_hour", IntegerType(), True), \
    StructField("date_minute", IntegerType(), True), \
    StructField("date_second", IntegerType(), True)
  ])

In [29]:
df = spark.read\
          .format("csv")\
          .options(**{'sep': '\t', 'header': 'true'})\
          .schema(schema)\
          .load("user_ratedmovies.dat")

In [30]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: string (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



In [31]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|      3|   1.0|      29|        10|     2006|       23|         17|         16|
|    75|     32|   4.5|      29|        10|     2006|       23|         23|         44|
|    75|    110|   4.0|      29|        10|     2006|       23|         30|          8|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|    163|   4.0|      29|        10|     2006|       23|         29|         30|
|    75|    165|   4.5|      29|        10|     2006|       23|         25|         15|
|    75|    173|   3.5|      29|        10|     2006|       23|         17|         37|
|    75|    296|   5.0|      29|        10|     2006|       23|         24|         49|
|    75|    353|   3.5|      29|

Но есть уже готовая обертка под все нужды

In [32]:
df = spark.read.csv(path='user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [33]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Так, а как сохранить? Лучше быть аккуратнее с overwrite, перезапишет весь указанный путь, append будет безопаснее

In [34]:
df.write.option("header",True)\
        .mode("overwrite")\
        .parquet('write_1.parquet')

А что с партицированием?

In [35]:
df.write.option("header",True)\
        .partitionBy('date_year')\
        .mode("overwrite")\
        .parquet('write_2.parquet')

Кстати, раз уж заговорили про схемы данных, то их можно задавать интереснее, например, под группированные данные

In [36]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



Со структурой можно работать и менять ее под ваши нужны

In [37]:
updatedDF = df2.withColumn("OtherInfo", 
    F.struct(F.col("id").alias("identifier"),
    F.col("gender").alias("gender"),
    F.col("salary").alias("salary"),
    F.when(F.col("salary").cast(IntegerType()) < 2000,"Low")
      .when(F.col("salary").cast(IntegerType()) < 4000,"Medium")
      .otherwise("High").alias("Salary_Grade")
  )).drop("id","gender","salary")

updatedDF.printSchema()
updatedDF.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- OtherInfo: struct (nullable = false)
 |    |-- identifier: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- salary: integer (nullable = true)
 |    |-- Salary_Grade: string (nullable = false)

+--------------------+------------------------+
|name                |OtherInfo               |
+--------------------+------------------------+
|{James, , Smith}    |{36636, M, 3100, Medium}|
|{Michael, Rose, }   |{40288, M, 4300, High}  |
|{Robert, , Williams}|{42114, M, 1400, Low}   |
|{Maria, Anne, Jones}|{39192, F, 5500, High}  |
|{Jen, Mary, Brown}  |{, F, -1, Low}          |
+--------------------+------------------------+



Что мы там сделали????

1) Создали новую структуру данных OtherInfo

2) Передали туда id (переименовав столбец), gender, salary

3) Создали столбец Salary_grade из условий

4) удалили id, gender, salary из старой структуры

Есть и еще структуры данных!

In [7]:
from pyspark.sql.types import ArrayType, MapType

In [39]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

In [40]:
structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

In [41]:
df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- hobbies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: integer
 |    |-- value: string (valueContainsNull = true)

+--------------------+-----------------+----------------+
|name                |hobbies          |properties      |
+--------------------+-----------------+----------------+
|{James, , Smith}    |[car, volleyball]|{4 -> d, 1 -> a}|
|{Michael, Rose, }   |[car, football]  |{2 -> b}        |
|{Robert, , Williams}|[box, music]     |{3 -> c}        |
+--------------------+-----------------+----------------+



In [42]:
df3.select('properties').collect()

[Row(properties={4: 'd', 1: 'a'}),
 Row(properties={2: 'b'}),
 Row(properties={3: 'c'})]

**Описание данных**

Общее описание данных

In [43]:
df = spark.read.csv(path='user_ratedmovies.dat', sep='\t', header=True, inferSchema=True, schema=None)

In [44]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|           userID|           movieID|            rating|          date_day|        date_month|        date_year|         date_hour|       date_minute|       date_second|
+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|           855598|            855598|            855598|            855598|            855598|           855598|            855598|            855598|            855598|
|   mean|35190.83255103448| 8710.179402008887| 3.437945156487042|15.568923723524366|  6.54079485926802|2005.323717446745|12.124531614145896|29.645382527775894|29.510222090280717|
| stddev|20385.00334699137|14446.852908494242|1.0025608721610357| 8.951201361275226|3.5063994259089704|2.

In [45]:
df.summary().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|summary|           userID|           movieID|            rating|          date_day|        date_month|        date_year|         date_hour|       date_minute|       date_second|
+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+
|  count|           855598|            855598|            855598|            855598|            855598|           855598|            855598|            855598|            855598|
|   mean|35190.83255103448| 8710.179402008887| 3.437945156487042|15.568923723524366|  6.54079485926802|2005.323717446745|12.124531614145896|29.645382527775894|29.510222090280717|
| stddev|20385.00334699137|14446.852908494242|1.0025608721610357| 8.951201361275226|3.5063994259089704|2.

Количество записей

In [46]:
df.count()

855598

Количество партиций

In [47]:
df.rdd.getNumPartitions()

2

 Менять число партиций можно, все как с rdd

In [48]:
df = df.repartition(5)

In [49]:
df.rdd.getNumPartitions()

5

In [50]:
df = df.coalesce(2)

In [51]:
df.rdd.getNumPartitions()

2

**Различные методы**

Ну теперь давайте тыкать 

Удаляем дубликаты и помним, что есть actions и transformations, count заставит все сделать

In [52]:
df_without_duplicates = df.drop_duplicates()

Есть alias

In [53]:
df_without_duplicates = df.dropDuplicates()

Как удалить дубликаты по отдельным колонкам?

In [54]:
df_without_duplicates = df.drop_duplicates(['userID', 'rating'])

In [55]:
df_without_duplicates.count()

18847

In [56]:
df_without_duplicates.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 21712|    534|   5.0|      10|         1|     1999|       21|         32|         12|
| 19519|   3408|   3.0|      21|         2|     2006|        1|         39|         13|
|  1047|    608|   3.0|      10|         4|     2008|       10|         32|          0|
| 28962|   1222|   4.0|      26|         5|     2005|       20|         19|          2|
| 18718|   5481|   2.0|      17|         9|     2008|       17|         17|         12|
|  4944|    924|   5.0|      18|         9|     2007|       22|         13|         38|
| 16862|   8638|   4.5|      10|         2|     2006|        5|         10|          4|
| 34145|   6365|   0.5|      21|         1|     2008|       22|         56|         24|
| 26243|   2174|   5.0|      19|

Корреляции

In [57]:
df.corr('rating', 'date_day')

0.016638388440497295

In [58]:
df.corr('rating', 'date_hour')

-0.012518740192686183

In [59]:
df.corr('rating', 'date_year')

-0.002129926269966018

Как закинуть данные в любимый pandas?

 Самый простой вариант - встроенный метод, но он жутко медленный при существенном объеме данных. Лучше сохранить паркет и считать через pd.read_parquet()

In [8]:
import pandas as pd

In [61]:
%%timeit 

pandas_df = df.toPandas()

7.37 s ± 1.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
pandas_df = df.toPandas()
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,38914,1732,4.0,8,2,2008,11,36,1
1,6393,2071,3.5,4,1,2006,21,28,23
2,32747,38061,4.0,5,4,2008,15,55,49
3,4352,30812,3.0,2,3,2006,22,34,12
4,37934,27674,3.5,8,10,2007,19,30,43
...,...,...,...,...,...,...,...,...,...
855593,46641,4022,4.5,9,3,2008,13,12,56
855594,57835,1438,2.5,1,8,2006,22,56,31
855595,46441,45447,2.0,28,5,2006,2,14,16
855596,65403,915,4.0,6,11,2005,6,30,56


In [63]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855598 entries, 0 to 855597
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userID       855598 non-null  int32  
 1   movieID      855598 non-null  int32  
 2   rating       855598 non-null  float64
 3   date_day     855598 non-null  int32  
 4   date_month   855598 non-null  int32  
 5   date_year    855598 non-null  int32  
 6   date_hour    855598 non-null  int32  
 7   date_minute  855598 non-null  int32  
 8   date_second  855598 non-null  int32  
dtypes: float64(1), int32(8)
memory usage: 32.6 MB


In [64]:
%%timeit 

pandas_df = pd.read_parquet('write_1.parquet')

33.7 ms ± 5.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
pandas_df = pd.read_parquet('write_1.parquet')
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30
...,...,...,...,...,...,...,...,...,...
855593,71534,44555,4.0,3,12,2007,3,5,38
855594,71534,46578,4.0,3,12,2007,2,56,44
855595,71534,48516,4.5,3,12,2007,2,53,46
855596,71534,61075,5.0,10,10,2008,9,56,5


In [66]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855598 entries, 0 to 855597
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   userID       855598 non-null  int32  
 1   movieID      855598 non-null  int32  
 2   rating       855598 non-null  float64
 3   date_day     855598 non-null  int32  
 4   date_month   855598 non-null  int32  
 5   date_year    855598 non-null  int32  
 6   date_hour    855598 non-null  int32  
 7   date_minute  855598 non-null  int32  
 8   date_second  855598 non-null  int32  
dtypes: float64(1), int32(8)
memory usage: 32.6 MB


Как говорили на лекции, может все упасть например тут (когда делаем toPandas). Как перейти к итератору?

prefetchPartitions - подготавливать ли следующую партию данных, пока не запросили

In [67]:
iter_df = df.toLocalIterator()

In [68]:
row = iter_df.send(None)

In [69]:
row

Row(userID=38914, movieID=1732, rating=4.0, date_day=8, date_month=2, date_year=2008, date_hour=11, date_minute=36, date_second=1)

In [70]:
row.asDict()

{'userID': 38914,
 'movieID': 1732,
 'rating': 4.0,
 'date_day': 8,
 'date_month': 2,
 'date_year': 2008,
 'date_hour': 11,
 'date_minute': 36,
 'date_second': 1}

Отсюда идея: можно вытягивать данные по 1 записи и записывать в датафрейм. Долго, но зато отработает.

In [71]:
iter_df = df.toLocalIterator()

In [74]:
list_of_rows = [value for value in iter_df]
print(len(list_of_rows))

0


In [75]:
df.columns

['userID',
 'movieID',
 'rating',
 'date_day',
 'date_month',
 'date_year',
 'date_hour',
 'date_minute',
 'date_second']

In [76]:
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

In [79]:
%%timeit

iter_df = df.toLocalIterator()
list_of_rows = [value for value in iter_df]
pandas_df = pd.DataFrame(list_of_rows, columns=df.columns)

7.97 s ± 1.41 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
pandas_df

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second


In [81]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       0 non-null      object
 1   movieID      0 non-null      object
 2   rating       0 non-null      object
 3   date_day     0 non-null      object
 4   date_month   0 non-null      object
 5   date_year    0 non-null      object
 6   date_hour    0 non-null      object
 7   date_minute  0 non-null      object
 8   date_second  0 non-null      object
dtypes: object(9)
memory usage: 0.0+ bytes


**Show**

In [82]:
df.show(10)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 38914|   1732|   4.0|       8|         2|     2008|       11|         36|          1|
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|
| 32747|  38061|   4.0|       5|         4|     2008|       15|         55|         49|
|  4352|  30812|   3.0|       2|         3|     2006|       22|         34|         12|
| 37934|  27674|   3.5|       8|        10|     2007|       19|         30|         43|
| 30840|  34148|   3.5|       6|        10|     2005|        1|         38|         58|
|  5954|   4007|   4.0|       8|         1|     2006|       22|          5|          2|
| 26243|    223|   3.0|      19|         3|     2006|        4|          4|         30|
|  3600|   3552|   3.5|      13|

Обрезаем до 2 символов

In [83]:
df.show(10, truncate=2)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    38|     17|    4.|       8|         2|       20|       11|         36|          1|
|    63|     20|    3.|       4|         1|       20|       21|         28|         23|
|    32|     38|    4.|       5|         4|       20|       15|         55|         49|
|    43|     30|    3.|       2|         3|       20|       22|         34|         12|
|    37|     27|    3.|       8|        10|       20|       19|         30|         43|
|    30|     34|    3.|       6|        10|       20|        1|         38|         58|
|    59|     40|    4.|       8|         1|       20|       22|          5|          2|
|    26|     22|    3.|      19|         3|       20|        4|          4|         30|
|    36|     35|    3.|      13|

вертикальное отображение

In [84]:
df.show(10, vertical=True)

-RECORD 0------------
 userID      | 38914 
 movieID     | 1732  
 rating      | 4.0   
 date_day    | 8     
 date_month  | 2     
 date_year   | 2008  
 date_hour   | 11    
 date_minute | 36    
 date_second | 1     
-RECORD 1------------
 userID      | 6393  
 movieID     | 2071  
 rating      | 3.5   
 date_day    | 4     
 date_month  | 1     
 date_year   | 2006  
 date_hour   | 21    
 date_minute | 28    
 date_second | 23    
-RECORD 2------------
 userID      | 32747 
 movieID     | 38061 
 rating      | 4.0   
 date_day    | 5     
 date_month  | 4     
 date_year   | 2008  
 date_hour   | 15    
 date_minute | 55    
 date_second | 49    
-RECORD 3------------
 userID      | 4352  
 movieID     | 30812 
 rating      | 3.0   
 date_day    | 2     
 date_month  | 3     
 date_year   | 2006  
 date_hour   | 22    
 date_minute | 34    
 date_second | 12    
-RECORD 4------------
 userID      | 37934 
 movieID     | 27674 
 rating      | 3.5   
 date_day    | 8     
 date_mont

**Select**

В PySpark функция select() используется для выбора одного, нескольких столбцов по индексу, всех столбцов из списка и вложенных столбцов из фрейма данных. Функция PySpark select() является функцией преобразования, поэтому она возвращает новый фрейм данных с выбранными столбцами.

In [85]:
df.columns

['userID',
 'movieID',
 'rating',
 'date_day',
 'date_month',
 'date_year',
 'date_hour',
 'date_minute',
 'date_second']

Упс, pandas-style тут не приветствуется

In [86]:
df.userID.show(5)

TypeError: ignored

In [87]:
df.select('userID').show(5)

+------+
|userID|
+------+
| 16592|
| 16592|
| 16592|
| 16592|
| 16592|
+------+
only showing top 5 rows



Куча вариантов, выбирайте любой

In [88]:
df.select('userID', 'rating').show(5)

+------+------+
|userID|rating|
+------+------+
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
+------+------+
only showing top 5 rows



In [89]:
df.select(['userID', 'rating']).show(5)

+------+------+
|userID|rating|
+------+------+
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
+------+------+
only showing top 5 rows



In [90]:
df.select(df.userID,df.rating).show(5)

+------+------+
|userID|rating|
+------+------+
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
+------+------+
only showing top 5 rows



In [91]:
df.select(df['userID'],df['rating']).show(5)

+------+------+
|userID|rating|
+------+------+
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
+------+------+
only showing top 5 rows



In [92]:
df.select(F.col("userID"), F.col("rating")).show(5)

+------+------+
|userID|rating|
+------+------+
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
| 13216|   4.5|
+------+------+
only showing top 5 rows



можно налету переименовать столбец

In [93]:
df.select(df.userID, df.rating.alias('mark')).show(5)

+------+----+
|userID|mark|
+------+----+
| 13216| 4.5|
| 13216| 4.5|
| 13216| 4.5|
| 13216| 4.5|
| 13216| 4.5|
+------+----+
only showing top 5 rows



In [94]:
#регулярки
df.select(df.colRegex("`d+.*y`")).show(5)

+--------+
|date_day|
+--------+
|      19|
|      19|
|      19|
|      19|
|      19|
+--------+
only showing top 5 rows



примеры с другим датафреймом, где структура сложнее

In [95]:
structureData = [
    (("James","","Smith"),"36636","M",3100),
    (("Michael","Rose",""),"40288","M",4300),
    (("Robert","","Williams"),"42114","M",1400),
    (("Maria","Anne","Jones"),"39192","F",5500),
    (("Jen","Mary","Brown"),"","F",-1)
  ]
structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+-----+------+------+
|name                |id   |gender|salary|
+--------------------+-----+------+------+
|{James, , Smith}    |36636|M     |3100  |
|{Michael, Rose, }   |40288|M     |4300  |
|{Robert, , Williams}|42114|M     |1400  |
|{Maria, Anne, Jones}|39192|F     |5500  |
|{Jen, Mary, Brown}  |     |F     |-1    |
+--------------------+-----+------+------+



In [96]:
df2.select('name').show(5)

+--------------------+
|                name|
+--------------------+
|    {James, , Smith}|
|   {Michael, Rose, }|
|{Robert, , Williams}|
|{Maria, Anne, Jones}|
|  {Jen, Mary, Brown}|
+--------------------+



In [97]:
df2.select('name.lastname').show(5)

+--------+
|lastname|
+--------+
|   Smith|
|        |
|Williams|
|   Jones|
|   Brown|
+--------+



In [98]:
df2.select('name.firstname', 'name.lastname').show(5)

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|        |
|   Robert|Williams|
|    Maria|   Jones|
|      Jen|   Brown|
+---------+--------+



**withColumn**

PySpark withColumn() - это функция преобразования (transform), которая используется для изменения значения, преобразования типа данных существующего столбца, создания нового столбца и многого другого. Поговорим о часто используемых операциях со столбцами данных PySpark, используя примеры.

In [99]:
df.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Меняем тип данных

In [100]:
df.withColumn("date_month", F.col("date_month").cast("String")).printSchema()

root
 |-- userID: integer (nullable = true)
 |-- movieID: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- date_day: integer (nullable = true)
 |-- date_month: string (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- date_hour: integer (nullable = true)
 |-- date_minute: integer (nullable = true)
 |-- date_second: integer (nullable = true)



Модифицировать столбец/создать новый

In [101]:
df.withColumn("rating_x_10",F.col("rating") * 10).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|rating_x_10|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----------+
| 38914|   1732|   4.0|       8|         2|     2008|       11|         36|          1|       40.0|
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|       35.0|
| 32747|  38061|   4.0|       5|         4|     2008|       15|         55|         49|       40.0|
|  4352|  30812|   3.0|       2|         3|     2006|       22|         34|         12|       30.0|
| 37934|  27674|   3.5|       8|        10|     2007|       19|         30|         43|       35.0|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----------+
only showing top 5 rows



Делаем 2 константных столбца

In [102]:
df.withColumn('fix_1', F.lit(1)).withColumn('fix_2', F.lit(2)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----+-----+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|fix_1|fix_2|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----+-----+
| 38914|   1732|   4.0|       8|         2|     2008|       11|         36|          1|    1|    2|
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|    1|    2|
| 32747|  38061|   4.0|       5|         4|     2008|       15|         55|         49|    1|    2|
|  4352|  30812|   3.0|       2|         3|     2006|       22|         34|         12|    1|    2|
| 37934|  27674|   3.5|       8|        10|     2007|       19|         30|         43|    1|    2|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+-----+-----+
only showing top 5 rows



**withColumnsRenamed**

Предыдущий вариант не давал возможности переименовать столбцы, это можно сделать иначе

In [103]:
df.withColumnRenamed('rating', 'mark').show(5)

+------+-------+----+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|mark|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+----+--------+----------+---------+---------+-----------+-----------+
| 38914|   1732| 4.0|       8|         2|     2008|       11|         36|          1|
|  6393|   2071| 3.5|       4|         1|     2006|       21|         28|         23|
| 32747|  38061| 4.0|       5|         4|     2008|       15|         55|         49|
|  4352|  30812| 3.0|       2|         3|     2006|       22|         34|         12|
| 37934|  27674| 3.5|       8|        10|     2007|       19|         30|         43|
+------+-------+----+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



**filter (where) и иные филтрации**

Функция PySpark filter() используется для фильтрации строк из RDD / DataFrame на основе заданного условия или выражения SQL, вы также можете использовать предложение where() вместо filter() обе эти функции работают аналогично.

1 условие

In [104]:
df.filter(df.rating == 5.0).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 35105|  41569|   5.0|      31|        12|     2005|        7|         31|         38|
| 30977|  45950|   5.0|      10|         7|     2006|        8|         16|          0|
| 36969|   1251|   5.0|       7|         5|     2006|       22|         35|         15|
|  7252|    296|   5.0|      29|        11|     2006|       19|         39|         34|
|  5917|   3196|   5.0|      12|         1|     2005|       10|         24|          6|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [105]:
df.filter(~(df.rating == 5.0)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 38914|   1732|   4.0|       8|         2|     2008|       11|         36|          1|
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|
| 32747|  38061|   4.0|       5|         4|     2008|       15|         55|         49|
|  4352|  30812|   3.0|       2|         3|     2006|       22|         34|         12|
| 37934|  27674|   3.5|       8|        10|     2007|       19|         30|         43|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [106]:
df.filter('rating = 5').show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 35105|  41569|   5.0|      31|        12|     2005|        7|         31|         38|
| 30977|  45950|   5.0|      10|         7|     2006|        8|         16|          0|
| 36969|   1251|   5.0|       7|         5|     2006|       22|         35|         15|
|  7252|    296|   5.0|      29|        11|     2006|       19|         39|         34|
|  5917|   3196|   5.0|      12|         1|     2005|       10|         24|          6|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



Несколько условий

In [107]:
df.filter((df.rating == 5.0) & (df.date_year == 2006)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 36969|   1251|   5.0|       7|         5|     2006|       22|         35|         15|
| 22278|   1732|   5.0|       5|         3|     2006|       16|         27|          7|
|  2853|    296|   5.0|      10|        11|     2006|       13|         52|         53|
|  4895|  27728|   5.0|       1|         1|     2006|        4|         15|          2|
| 19532|   2959|   5.0|      20|         2|     2006|        1|         51|         40|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [108]:
df.filter('(rating = 5.0) and (date_year = 2006)').show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
| 36969|   1251|   5.0|       7|         5|     2006|       22|         35|         15|
| 22278|   1732|   5.0|       5|         3|     2006|       16|         27|          7|
|  2853|    296|   5.0|      10|        11|     2006|       13|         52|         53|
|  4895|  27728|   5.0|       1|         1|     2006|        4|         15|          2|
| 19532|   2959|   5.0|      20|         2|     2006|        1|         51|         40|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



In [109]:
df.filter('(rating = 5.0) and (userID between 70 and 80)').show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    78|   1214|   5.0|       7|         5|     2004|       23|          6|         34|
|    78|     32|   5.0|       7|         5|     2004|       23|         32|         18|
|    78|   2502|   5.0|       7|         5|     2004|       23|         38|         29|
|    78|    750|   5.0|       7|         5|     2004|       23|          3|         45|
|    78|   1232|   5.0|       7|         5|     2004|       23|         29|         49|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



фильтр по списку значений из list

In [110]:
years = [2006, 2007]

In [111]:
df.filter(df.date_year.isin(years)).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|  6393|   2071|   3.5|       4|         1|     2006|       21|         28|         23|
| 37934|  27674|   3.5|       8|        10|     2007|       19|         30|         43|
| 25038|   3250|   3.0|      30|        12|     2007|        8|         34|         29|
| 26243|    223|   3.0|      19|         3|     2006|        4|          4|         30|
| 15311|   1411|   3.0|      22|        12|     2007|       22|         42|          4|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
only showing top 5 rows



 проверим

In [112]:
df.filter(df.date_year.isin(years))\
  .select('date_year')\
  .dropDuplicates()\
  .collect()

[Row(date_year=2007), Row(date_year=2006)]

создадим игрушечный датайфрейм для текстовых столбцов

In [113]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5,"Rames Black"), (6, 'Albus Torch'),
     (7, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

In [114]:
df2.show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|    Rames Black|
|  6|    Albus Torch|
|  7|        Fred Tf|
+---+---------------+



In [115]:
df2.filter('name like "R%"').show()

+---+---------------+
| id|           name|
+---+---------------+
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|    Rames Black|
+---+---------------+



In [116]:
df2.filter(df2.name.startswith('R')).show()

+---+---------------+
| id|           name|
+---+---------------+
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|    Rames Black|
+---+---------------+



In [117]:
df2.filter(df2.name.endswith('Tf')).show()

+---+-------+
| id|   name|
+---+-------+
|  7|Fred Tf|
+---+-------+



In [118]:
df2.filter(df2.name.contains('Wil')).show()

+---+---------------+
| id|           name|
+---+---------------+
|  3|Robert Williams|
+---+---------------+



Бывает, что у нас внутри датафрейма есть массив и с ним что-то хочется сделать

In [119]:
arrayStructureSchema = StructType([
    StructField('name', StructType([
       StructField('firstname', StringType(), True),
       StructField('middlename', StringType(), True),
       StructField('lastname', StringType(), True)
       ])),
       StructField('hobbies', ArrayType(StringType()), True),
       StructField('properties', MapType(IntegerType(),StringType()), True)
    ])

structureData = [
    (("James","","Smith"), ['car', 'volleyball'], {1: 'a', 4: 'd'}),
    (("Michael","Rose",""), ['car', 'football'], {2: 'b'}),
    (("Robert","","Williams"), ['box', 'music'], {3: 'c'})
  ]

df3 = spark.createDataFrame(data=structureData,schema=arrayStructureSchema)
df3.printSchema()
df3.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- hobbies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- properties: map (nullable = true)
 |    |-- key: integer
 |    |-- value: string (valueContainsNull = true)

+--------------------+-----------------+----------------+
|name                |hobbies          |properties      |
+--------------------+-----------------+----------------+
|{James, , Smith}    |[car, volleyball]|{4 -> d, 1 -> a}|
|{Michael, Rose, }   |[car, football]  |{2 -> b}        |
|{Robert, , Williams}|[box, music]     |{3 -> c}        |
+--------------------+-----------------+----------------+



In [120]:
df3.filter(F.array_contains(df3.hobbies,"football")) \
    .show()  

+-----------------+---------------+----------+
|             name|        hobbies|properties|
+-----------------+---------------+----------+
|{Michael, Rose, }|[car, football]|  {2 -> b}|
+-----------------+---------------+----------+



**Сортировка**

сделаем еще фильтрацию, чтобы увидеть результат (orderBy тут аналог)

In [121]:
df.filter(df.userID == 75).sort(df.date_minute, df.rating.desc()).show(20)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+
|    75|   1304|   2.5|      29|        10|     2006|       23|         16|         56|
|    75|    420|   2.0|      29|        10|     2006|       23|         16|         42|
|    75|   2011|   2.0|      29|        10|     2006|       23|         16|         39|
|    75|    160|   2.0|      29|        10|     2006|       23|         16|         52|
|    75|   2700|   4.5|      29|        10|     2006|       23|         17|         52|
|    75|    832|   4.5|      29|        10|     2006|       23|         17|         49|
|    75|   1374|   4.0|      29|        10|     2006|       23|         17|         20|
|    75|   1485|   4.0|      29|        10|     2006|       23|         17|         46|
|    75|    173|   3.5|      29|

**groupby**

Когда мы выполняем groupBy() в PySpark DataFrame, он возвращает объект GroupedData, который содержит следующие агрегатные функции:

min(), max(), mean(), count(), sum(), avg(), agg(), pivot() 

In [122]:
df.groupby('date_year').mean('rating').collect()

[Row(date_year=2003, avg(rating)=3.4279154852282083),
 Row(date_year=2007, avg(rating)=3.4367384383270894),
 Row(date_year=2006, avg(rating)=3.4294610550371356),
 Row(date_year=1997, avg(rating)=3.571018651362984),
 Row(date_year=2004, avg(rating)=3.3916603359500295),
 Row(date_year=1998, avg(rating)=3.5080568720379146),
 Row(date_year=2009, avg(rating)=3.369243156199678),
 Row(date_year=2001, avg(rating)=3.4490124195720484),
 Row(date_year=2005, avg(rating)=3.395790002433071),
 Row(date_year=2000, avg(rating)=3.5767733835530446),
 Row(date_year=2008, avg(rating)=3.486752231109465),
 Row(date_year=1999, avg(rating)=3.4925797284496367),
 Row(date_year=2002, avg(rating)=3.4140127388535033)]

мы уже умеем применять разные методы

In [123]:
df.groupby('date_year')\
  .mean('rating')\
  .sort('date_year')\
  .collect()

[Row(date_year=1997, avg(rating)=3.571018651362984),
 Row(date_year=1998, avg(rating)=3.5080568720379146),
 Row(date_year=1999, avg(rating)=3.4925797284496367),
 Row(date_year=2000, avg(rating)=3.5767733835530446),
 Row(date_year=2001, avg(rating)=3.4490124195720484),
 Row(date_year=2002, avg(rating)=3.4140127388535033),
 Row(date_year=2003, avg(rating)=3.4279154852282083),
 Row(date_year=2004, avg(rating)=3.3916603359500295),
 Row(date_year=2005, avg(rating)=3.395790002433071),
 Row(date_year=2006, avg(rating)=3.4294610550371356),
 Row(date_year=2007, avg(rating)=3.4367384383270894),
 Row(date_year=2008, avg(rating)=3.486752231109465),
 Row(date_year=2009, avg(rating)=3.369243156199678)]

In [124]:
df.filter(df.rating <= 2)\
  .groupby('date_year')\
  .count()\
  .withColumnRenamed('count', 'number')\
  .sort('date_year')\
  .collect()

[Row(date_year=1997, number=98),
 Row(date_year=1998, number=321),
 Row(date_year=1999, number=2270),
 Row(date_year=2000, number=4732),
 Row(date_year=2001, number=4793),
 Row(date_year=2002, number=5791),
 Row(date_year=2003, number=7595),
 Row(date_year=2004, number=10205),
 Row(date_year=2005, number=15982),
 Row(date_year=2006, number=23463),
 Row(date_year=2007, number=19792),
 Row(date_year=2008, number=15128),
 Row(date_year=2009, number=369)]

несколько колонок

In [125]:
df.groupBy("date_year", "date_month") \
  .mean("rating", "userID") \
  .sort('date_year', 'date_month') \
  .show()

+---------+----------+------------------+------------------+
|date_year|date_month|       avg(rating)|       avg(userID)|
+---------+----------+------------------+------------------+
|     1997|         9|3.7094972067039107| 35144.61452513967|
|     1997|        10|3.7762237762237763|34880.769230769234|
|     1997|        11| 3.360655737704918| 9451.180327868853|
|     1997|        12| 3.549618320610687| 20251.01526717557|
|     1998|         1|          3.765625|      22209.515625|
|     1998|         2|               3.3|           31662.4|
|     1998|         3| 3.238095238095238| 19582.04761904762|
|     1998|         4| 3.908333333333333|31813.866666666665|
|     1998|         5| 3.652054794520548|20393.520547945205|
|     1998|         6|             3.225|         27092.625|
|     1998|         7|3.5547550432276656|38376.249279538904|
|     1998|         8| 3.642857142857143|41151.357142857145|
|     1998|         9|3.1881533101045294| 46104.74912891986|
|     1998|        10|3.

Для того, чтобы делать несколько разных агрегаций и еще менять сразу имя столбца нужно немного изменить синтаксис

И стоит помнить, что персентили тут считаются приближенно

In [126]:
df.groupBy("date_year") \
    .agg(F.min("rating").alias("min_rating"), \
         F.mean("rating").alias("mean_rating"), \
         F.max("rating").alias("max_rating"),
         F.percentile_approx("rating", 0.5).alias("median")
         ) \
    .show()

+---------+----------+------------------+----------+------+
|date_year|min_rating|       mean_rating|max_rating|median|
+---------+----------+------------------+----------+------+
|     2003|       0.5|3.4279154852282083|       5.0|   3.5|
|     2007|       0.5|3.4367384383270894|       5.0|   3.5|
|     2006|       0.5|3.4294610550371356|       5.0|   3.5|
|     1997|       1.0| 3.571018651362984|       5.0|   4.0|
|     2004|       0.5|3.3916603359500295|       5.0|   3.5|
|     1998|       1.0|3.5080568720379146|       5.0|   4.0|
|     2009|       0.5| 3.369243156199678|       5.0|   3.5|
|     2001|       1.0|3.4490124195720484|       5.0|   4.0|
|     2005|       0.5| 3.395790002433071|       5.0|   3.5|
|     2000|       1.0|3.5767733835530446|       5.0|   4.0|
|     2008|       0.5| 3.486752231109465|       5.0|   3.5|
|     1999|       1.0|3.4925797284496367|       5.0|   4.0|
|     2002|       1.0|3.4140127388535033|       5.0|   4.0|
+---------+----------+------------------

Еще можно сделать pivot

In [127]:
df.groupBy('date_year')\
  .pivot('date_month')\
  .mean('rating')\
  .show(5)

+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|date_year|                 1|                 2|                 3|                 4|                 5|                 6|                 7|                 8|                 9|                10|                11|                12|
+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|     2003|3.4720922980979108| 3.333741687084354|3.4834824501032347|3.4579510703363914| 3.366686911630732|  3.55358230160341|3.5085277150739906| 3.447642637512812| 3.579355840124175| 3.165437788018433| 3.446676231468197| 3.440287081339713|
|     2007| 3.520191839855425| 3.3341516

**Join's**

Куда же без них. Что есть: INNER, LEFT OUTER, RIGHT OUTER, LEFT ANTI, LEFT SEMI, CROSS, SELF JOIN

Благодаря оптимизации в датафреймах уже все хорошо работает, спасибо catalist, но чудеса не вечны и плохой код/незнание данных все равно даст о себе знать

Сделаем для себя несколько таблиц, чтобы можно было экспериментировать

In [128]:
df_mean_user_rating = df.groupBy("userID")\
                        .mean('rating')\
                        .withColumnRenamed('avg(rating)', 'avg_rating_all')

df_mean_user_rating_year = df.groupby('userID', 'date_year')\
                             .mean('rating')\
                             .withColumnRenamed('avg(rating)', 'avg_rating_year')

In [129]:
df_mean_user_rating.printSchema()

df_mean_user_rating_year.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- avg_rating_all: double (nullable = true)

root
 |-- userID: integer (nullable = true)
 |-- date_year: integer (nullable = true)
 |-- avg_rating_year: double (nullable = true)



И давайте все в 1 блоке кода, чтобы не растягивать

In [130]:
df.join(df_mean_user_rating, on=df.userID == df_mean_user_rating.userID, how='inner')\
  .join(df_mean_user_rating_year, on=[df.userID == df_mean_user_rating_year.userID,
                                      df.date_year == df_mean_user_rating_year.date_year],
        how='inner')

AnalysisException: ignored

Надо удалить дублирующиеся столбцы

In [131]:
res_join = df.alias('t').join(df_mean_user_rating.alias('t1'), on=F.col('t.userID') == F.col('t1.userID'), how='inner')\
  .drop(F.col('t1.userID'))\
  .join(df_mean_user_rating_year.alias('t2'), on=[F.col('t.userID') == F.col('t2.userID'),
                                      F.col('t.date_year') == F.col('t2.date_year')],
        how='inner')\
  .drop(F.col('t2.userID'))\
  .drop(F.col('t2.date_year'))

In [132]:
res_join.show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|    avg_rating_all|   avg_rating_year|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
| 14570|   1653|   3.0|       2|         7|     2003|        2|         46|          1|2.7909407665505226| 2.634715025906736|
| 15846|   5902|   4.5|      20|         8|     2004|       21|         53|          3| 3.792682926829268|3.7694300518134716|
| 24171|  36517|   3.5|       5|         7|     2007|       17|         14|         47| 3.785140562248996| 3.818396226415094|
| 24171|   5445|   4.0|       3|         7|     2007|       15|         47|         37| 3.785140562248996| 3.818396226415094|
| 24171|    380|   4.0|       3|         7|     2007|       15|         42|         12| 3.785140562248996| 3.818396226

**union и unionAll**

Используются для объединения датафреймов с одинаковой структурой, используется union, так как unionAll с версии 2.0.0 более не используется

In [133]:
df1 = df.filter(df.date_year == 2006)
df2 = df.filter(df.date_year != 2006)

In [134]:
union_df = df1.union(df2)

print(df.count(), union_df.count())

855598 855598


Desclaimer: все по sql, надеюсь, помнят разницу между union и union all, когда union убирает дубликаты. Так вот pyspark ничего не удаляет, убрать дубликаты можно только через drop_duplicates, distinct

Также при union pyspark делает объединение по столбцам as is, не пытаясь понять, что в одном датафрейме нужный стобец на 1 позиции, а в другом он на 5. Для этого с версии 3.1 есть замечательный метод unionByName

**UDF - user defined functions**

из курса про rdd помним про map, тут тоже можно перегнать все в rdd и делать map, но можно и через udf. Стоит отметить, что при этом мы теряем возможность оптимизации и произодительность в dataframe, так как udf - black box для спарка.

Но зато эти функции переиспользуемы и их можно применять в sql запросах, как те же udf в oracle

In [135]:
def udf_example(rating):
    rating = rating * 20
    return rating

In [9]:
from pyspark.sql.functions import udf

In [137]:
my_udf = udf(lambda x: udf_example(x), DoubleType())

In [138]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', my_udf(F.col('rating')))\
  .show(5)

+------+-------+------+----------+
|userID|movieID|rating|rating_100|
+------+-------+------+----------+
| 24515|  54272|   4.0|      80.0|
|  3190|   6303|   3.5|      70.0|
|  8881|   2942|   3.0|      60.0|
| 30687|   4226|   4.0|      80.0|
| 36292|   3910|   1.0|      20.0|
+------+-------+------+----------+
only showing top 5 rows



Для тех, кто любит декораторы

In [139]:
@udf(returnType=DoubleType()) 
def udf_example_decorator(rating):
    rating = rating * 20
    return rating

In [140]:
df.select(['userID', 'movieID', 'rating'])\
  .withColumn('rating_100', udf_example_decorator(F.col('rating')))\
  .show(5)

+------+-------+------+----------+
|userID|movieID|rating|rating_100|
+------+-------+------+----------+
| 24515|  54272|   4.0|      80.0|
|  3190|   6303|   3.5|      70.0|
|  8881|   2942|   3.0|      60.0|
| 30687|   4226|   4.0|      80.0|
| 36292|   3910|   1.0|      20.0|
+------+-------+------+----------+
only showing top 5 rows



Зарегистрируем функцию для будущих примеров с sql

In [141]:
spark.udf.register("udf_example_decorator", udf_example_decorator)

<function __main__.udf_example_decorator(rating)>

**SQL**

Ну раз уж пошла такая тема, давайте рассмотрим, как можно сделать все при помощи любимого SQL

можно делать TempView и GlodalTempView, отличие в том, что обычный view будет жить, пока жива сессия спрака, а глобальная, пока жив sparkcontext

In [142]:
df.createOrReplaceTempView('df')

In [143]:
query = '''
select userID, movieID, rating, udf_example_decorator(rating) as rating_100
from
df
'''
spark.sql(query).show(5)

+------+-------+------+----------+
|userID|movieID|rating|rating_100|
+------+-------+------+----------+
| 24515|  54272|   4.0|      80.0|
|  3190|   6303|   3.5|      70.0|
|  8881|   2942|   3.0|      60.0|
| 30687|   4226|   4.0|      80.0|
| 36292|   3910|   1.0|      20.0|
+------+-------+------+----------+
only showing top 5 rows



Ну и наш join

In [144]:
df_mean_user_rating.createOrReplaceTempView('df_mean_user_rating')
df_mean_user_rating_year.createOrReplaceTempView('df_mean_user_rating_year')

In [145]:
query = '''
select t.*, t1.avg_rating_all, t2.avg_rating_year
from
df t, df_mean_user_rating t1, df_mean_user_rating_year t2
where
    t.userID = t1.userID and
    t.userID = t2.userID and
    t.date_year = t2.date_year
'''
spark.sql(query).show(5)

+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
|userID|movieID|rating|date_day|date_month|date_year|date_hour|date_minute|date_second|    avg_rating_all|   avg_rating_year|
+------+-------+------+--------+----------+---------+---------+-----------+-----------+------------------+------------------+
| 14570|   1653|   3.0|       2|         7|     2003|        2|         46|          1|2.7909407665505226| 2.634715025906736|
| 15846|   5902|   4.5|      20|         8|     2004|       21|         53|          3| 3.792682926829268|3.7694300518134716|
| 24171|  36517|   3.5|       5|         7|     2007|       17|         14|         47| 3.785140562248996| 3.818396226415094|
| 24171|   5445|   4.0|       3|         7|     2007|       15|         47|         37| 3.785140562248996| 3.818396226415094|
| 24171|    380|   4.0|       3|         7|     2007|       15|         42|         12| 3.785140562248996| 3.818396226

**fill() и fillna()**

Оба метода идентичны, заполняют пропуски

In [10]:
import numpy as np

In [147]:
data2 = [(2,"Michael Rose"),(3,"Robert Williams"),
     (4,"Rames Rose"),(5, None), (6, None),
     (None, 'Fred Tf')
  ]
df2 = spark.createDataFrame(data2, ['id', 'name'])

А где пропуски?

In [148]:
df2.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df2.columns]).show()

+---+----+
| id|name|
+---+----+
|  1|   2|
+---+----+



In [149]:
df2.show()

+----+---------------+
|  id|           name|
+----+---------------+
|   2|   Michael Rose|
|   3|Robert Williams|
|   4|     Rames Rose|
|   5|           null|
|   6|           null|
|null|        Fred Tf|
+----+---------------+



In [150]:
df2.fillna({'id': 0}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|           null|
|  6|           null|
|  0|        Fred Tf|
+---+---------------+



In [151]:
df2.fillna({'id': 0, 'name': 'Unknown'}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|        Unknown|
|  6|        Unknown|
|  0|        Fred Tf|
+---+---------------+



Аналогично

In [152]:
df2.na.fill({'id': 0, 'name': 'Unknown'}).show()

+---+---------------+
| id|           name|
+---+---------------+
|  2|   Michael Rose|
|  3|Robert Williams|
|  4|     Rames Rose|
|  5|        Unknown|
|  6|        Unknown|
|  0|        Fred Tf|
+---+---------------+



Немного поговорим о том, как делаеть подвыборки

**sample и sampleBy**

Не забываем про возможную некоторую недетерминированность

In [11]:
import pyspark.sql.functions as F

In [12]:
trans_data = spark.read.parquet('/content/drive/MyDrive/spark_transactions.parquet')
trans_data = trans_data.withColumn('target', F.when(F.col('IsFraud')=='No', 0).otherwise(1))

In [13]:
trans_data.count()

24386900

In [14]:
trans_data.select(F.mean(F.col('target'))).show()

+--------------------+
|         avg(target)|
+--------------------+
|0.001220204290008...|
+--------------------+



In [15]:
trans_data.select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+--------+
|target|   count|
+------+--------+
|     1|   29757|
|     0|24357143|
+------+--------+



In [16]:
trans_data_simple = trans_data.sample(withReplacement=False, fraction=0.1, seed=3)
print(trans_data_simple.count())
trans_data_simple.select(F.mean(F.col('target'))).show()

2439262
+-------------------+
|        avg(target)|
+-------------------+
|0.00120897222192614|
+-------------------+



Стратификация

Тут важно понимать, что это не scikit-learn и стратификация предполагает, что вы по какому-то полю можете выбрать определенную долю наблюдений по его значениям

In [159]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+-----+
|target|count|
+------+-----+
|     1|29757|
+------+-----+



Не указали какой-то ключ - его доля будет 0

In [160]:
trans_data.sampleBy(F.col('target'), fractions={1: 1.0, 0: 0.1}, seed=0)\
          .select('target')\
          .groupBy('target')\
          .count()\
          .show()

+------+-------+
|target|  count|
+------+-------+
|     1|  29757|
|     0|2436333|
+------+-------+



Оконные функции **F.func().over(Window.partitionBy().orderBy())**

In [17]:
from pyspark.sql.window import Window

In [162]:
trans_data_simple.select('User', 'Card', 'Year', 'Month', 'Day')\
.withColumn('rn_1', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc())))\
.withColumn('rn_2', F.row_number().over(Window.partitionBy('User', 'Card').orderBy(F.col('Year').asc(), F.col('Month').desc())))\
.withColumn('mean', F.mean('Day').over(Window.partitionBy('User', 'Card', 'Month')))\
.withColumn('lag', F.lag('Day').over(Window.partitionBy('User', 'Card', 'Month').orderBy(F.col('Year').asc(),
                                                                                         F.col('Month').desc(),
                                                                                         F.col('Day').desc())))\
.show()

+----+----+----+-----+---+----+----+------------------+----+
|User|Card|Year|Month|Day|rn_1|rn_2|              mean| lag|
+----+----+----+-----+---+----+----+------------------+----+
|   0|   1|2015|    1| 24|  19|  37|              13.6|null|
|   0|   1|2015|    1| 16|  18|  36|              13.6|  24|
|   0|   1|2016|    1|  8|  41|  57|              13.6|  16|
|   0|   1|2016|    1|  5|  40|  56|              13.6|   8|
|   0|   1|2016|    1|  3|  39|  55|              13.6|   5|
|   0|   1|2016|    1|  1|  38|  54|              13.6|   3|
|   0|   1|2018|    1| 27|  74|  88|              13.6|   1|
|   0|   1|2018|    1|  5|  73|  87|              13.6|  27|
|   0|   1|2020|    1| 26| 102| 105|              13.6|   5|
|   0|   1|2020|    1| 21| 101| 104|              13.6|  26|
|   0|   1|2015|    2| 15|  20|  35|15.666666666666666|null|
|   0|   1|2016|    2|  7|  42|  53|15.666666666666666|  15|
|   0|   1|2018|    2| 23|  76|  86|15.666666666666666|   7|
|   0|   1|2018|    2|  

Сборка последовательностей через **collect_list()**

In [163]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)
 |-- target: integer (nullable = false)



In [164]:
trans_data_simple_seq = trans_data_simple.select('User', 'Card', F.struct('Amount', 'Day').alias('sequence'))

In [165]:
trans_data_simple_seq.show()

+----+----+-------------+
|User|Card|     sequence|
+----+----+-------------+
|   0|   0|{$162.39, 11}|
|   0|   0|{$159.72, 17}|
|   0|   0| {$92.30, 18}|
|   0|   0|{$168.11, 19}|
|   0|   0|{$119.69, 20}|
|   0|   0| {$48.34, 21}|
|   0|   0|{$141.70, 28}|
|   0|   0| {$40.58, 29}|
|   0|   0|  {$16.87, 5}|
|   0|   0|  {$1.84, 18}|
|   0|   0| {$57.00, 21}|
|   0|   0| {$66.00, 24}|
|   0|   0| {$35.82, 25}|
|   0|   0| {$77.85, 30}|
|   0|   0| {$96.00, 31}|
|   0|   0|  {$30.50, 2}|
|   0|   0|  {$75.72, 4}|
|   0|   0| {$122.64, 8}|
|   0|   0|{$127.33, 18}|
|   0|   0| {$83.17, 20}|
+----+----+-------------+
only showing top 20 rows



In [166]:
trans_data_simple_seq = trans_data_simple_seq.groupBy('User', 'Card')\
                                             .agg(F.collect_list('sequence').alias('sequence'))

In [167]:
trans_data_simple_seq.show(truncate=50)

+----+----+--------------------------------------------------+
|User|Card|                                          sequence|
+----+----+--------------------------------------------------+
|   0|   1|[{$20.61, 29}, {$42.80, 6}, {$13.49, 6}, {$149....|
|   1|   0|                      [{$60.86, 18}, {$29.18, 18}]|
|   1|   1|[{$120.00, 2}, {$24.63, 10}, {$67.21, 13}, {$14...|
|   1|   3|[{$12.90, 25}, {$2.72, 14}, {$55.14, 3}, {$96.0...|
|   1|   4|[{$90.43, 8}, {$10.77, 16}, {$0.67, 26}, {$14.1...|
|   3|   0|[{$141.16, 9}, {$340.00, 10}, {$400.00, 15}, {$...|
|   4|   0|[{$42.88, 2}, {$464.35, 12}, {$103.37, 14}, {$1...|
|   5|   1|[{$113.45, 1}, {$18.80, 15}, {$16.82, 20}, {$22...|
|   6|   1|[{$433.47, 5}, {$34.39, 10}, {$34.96, 12}, {$-4...|
|   8|   1|[{$32.67, 2}, {$16.20, 23}, {$18.71, 27}, {$47....|
|   8|   3|[{$21.24, 5}, {$19.27, 12}, {$122.29, 14}, {$13...|
|  13|   0|[{$1128.96, 18}, {$16.34, 22}, {$314.23, 7}, {$...|
|  13|   1|[{$16.81, 16}, {$43.04, 18}, {$17.67, 23}, {

А что с этим делать дальше? Можно перейти к RDD и построчно обрабатывать

In [73]:
row = trans_data_simple_seq.rdd.take(1)

NameError: ignored

In [72]:
row

NameError: ignored

In [170]:
row[0]['sequence']

[Row(Amount='$20.61', Day=29),
 Row(Amount='$42.80', Day=6),
 Row(Amount='$13.49', Day=6),
 Row(Amount='$149.22', Day=17),
 Row(Amount='$172.21', Day=1),
 Row(Amount='$58.68', Day=5),
 Row(Amount='$33.22', Day=25),
 Row(Amount='$154.21', Day=29),
 Row(Amount='$10.47', Day=9),
 Row(Amount='$6.84', Day=4),
 Row(Amount='$87.28', Day=1),
 Row(Amount='$81.00', Day=15),
 Row(Amount='$-81.00', Day=15),
 Row(Amount='$39.80', Day=25),
 Row(Amount='$5.22', Day=1),
 Row(Amount='$14.08', Day=2),
 Row(Amount='$30.24', Day=13),
 Row(Amount='$66.30', Day=16),
 Row(Amount='$12.31', Day=24),
 Row(Amount='$120.10', Day=15),
 Row(Amount='$38.08', Day=1),
 Row(Amount='$63.82', Day=31),
 Row(Amount='$151.48', Day=6),
 Row(Amount='$50.01', Day=15),
 Row(Amount='$45.55', Day=24),
 Row(Amount='$103.04', Day=2),
 Row(Amount='$12.95', Day=13),
 Row(Amount='$10.26', Day=12),
 Row(Amount='$65.00', Day=13),
 Row(Amount='$1.49', Day=22),
 Row(Amount='$1.51', Day=10),
 Row(Amount='$24.14', Day=17),
 Row(Amount='$28.

**Домашнее задание**

Куда же без домашки, верно?

Есть данные по транзакциям клиентов, ваша задача состоит в анализе этих данных и подготовки к структуре, которая похожа на ту структуру, которая сейчас часто нами используется при построении моделей на транзакциях + промежуточные задания.

Не забудьте делать всякие show после каждого задания, чтобы было видно результат

**Файл spark_transactions.parquet можете забрать в папке с записями лекций**

**Важно**
В домашнем задании старайтесь использовать максимально dataframe api, а не sql запросы.

In [18]:
trans_data = spark.read.parquet('/content/drive/MyDrive/spark_transactions.parquet')

In [19]:
trans_data.count()

24386900

In [20]:
trans_data.show(5)

+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|User|Card|Year|Month|Day| Time| Amount|          UseChip| MerchantCity|MerchantState|    Zip| MCC|IsFraud|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|     La Verne|           CA|91750.0|5300|     No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|Monterey Park|           CA|91754.0|5411|     No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|Monterey Park|           CA|91754.0|5651|     No|
|   0|   0|2002|    9|  3|06:23|$104.71|Swipe Transaction|     La Verne|           CA|91750.0|5912|     No|
+----+----+----+-----+---+-----+-------+-----------------+-------------+-------------+-------+----+-------+
only showing top 5 rows



Посмотрим на схему данных

In [21]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: double (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)



Сколько в среднем транзакций у пользователя

In [22]:
trans_data.groupBy('User')\
          .agg(F.count('Amount').alias('tranz_cnt'))\
          .select(F.mean('tranz_cnt'))\
          .alias('avg_trans_cnt')\
          .show()

+--------------+
|avg(tranz_cnt)|
+--------------+
|      12193.45|
+--------------+



Сколько карт у пользователей в среднем

In [23]:
trans_data.groupBy('User', 'Card')\
          .agg(F.lit(1).alias('card_cnt'))\
          .groupBy('User')\
          .agg(F.sum('card_cnt').alias('card_cnt'))\
          .select(F.mean('card_cnt').alias('avg_card_cnt'))\
          .show()

+------------+
|avg_card_cnt|
+------------+
|      3.0695|
+------------+



Немного обработаем данные: Amount в float, из Time вытянем час транзакции и удалим исходный Time, Zip  к типу int

In [24]:
@udf(returnType=FloatType()) 
def amount_to_float(x):
    return float(x[1:])


trans_data = trans_data.withColumn("Amount", amount_to_float("Amount"))\
              .withColumn("Zip", F.col("Zip").cast("Int"))\
              .withColumn("Time_hour", F.hour(F.col("Time").cast("Timestamp")))\
              .drop('Time')

In [25]:
trans_data.show(5)

+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+---------+
|User|Card|Year|Month|Day|Amount|          UseChip| MerchantCity|MerchantState|  Zip| MCC|IsFraud|Time_hour|
+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+---------+
|   0|   0|2002|    9|  1|134.09|Swipe Transaction|     La Verne|           CA|91750|5300|     No|        6|
|   0|   0|2002|    9|  1| 38.48|Swipe Transaction|Monterey Park|           CA|91754|5411|     No|        6|
|   0|   0|2002|    9|  2|120.34|Swipe Transaction|Monterey Park|           CA|91754|5411|     No|        6|
|   0|   0|2002|    9|  2|128.95|Swipe Transaction|Monterey Park|           CA|91754|5651|     No|       17|
|   0|   0|2002|    9|  3|104.71|Swipe Transaction|     La Verne|           CA|91750|5912|     No|        6|
+----+----+----+-----+---+------+-----------------+-------------+-------------+-----+----+-------+---------+
only showing top 5 

In [26]:
trans_data.printSchema()

root
 |-- User: long (nullable = true)
 |-- Card: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- Day: long (nullable = true)
 |-- Amount: float (nullable = true)
 |-- UseChip: string (nullable = true)
 |-- MerchantCity: string (nullable = true)
 |-- MerchantState: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- MCC: long (nullable = true)
 |-- IsFraud: string (nullable = true)
 |-- Time_hour: integer (nullable = true)



Посчитайте количество транзакций по годам, учитывая только те транзакции, объем которых был больше 100

In [27]:
trans_data.filter(trans_data.Amount > 100)\
          .groupBy('Year')\
          .count()\
          .sort('Year')\
          .show()

+----+------+
|Year| count|
+----+------+
|1991|   266|
|1992|   680|
|1993|  1050|
|1994|  1533|
|1995|  2132|
|1996|  3168|
|1997|  5593|
|1998|  9009|
|1999| 13656|
|2000| 20816|
|2001| 30504|
|2002| 40944|
|2003| 54288|
|2004| 69846|
|2005| 87779|
|2006|105907|
|2007|121489|
|2008|137348|
|2009|150327|
|2010|165024|
+----+------+
only showing top 20 rows



Определите, есть ли пропуски в данных по каждому столбцу

In [28]:
trans_data.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in trans_data.columns]).show()

+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+---------+
|User|Card|Year|Month|Day|Amount|UseChip|MerchantCity|MerchantState|    Zip|MCC|IsFraud|Time_hour|
+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+---------+
|   0|   0|   0|    0|  0|     0|      0|           0|      2720821|2878135|  0|      0|        0|
+----+----+----+-----+---+------+-------+------------+-------------+-------+---+-------+---------+



Заполните пропуски исходя из типа данных

In [29]:
trans_data = trans_data.na.fill({'MerchantState': 'Unknown', 'Zip': -1})
trans_data.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in trans_data.columns]).show()

+----+----+----+-----+---+------+-------+------------+-------------+---+---+-------+---------+
|User|Card|Year|Month|Day|Amount|UseChip|MerchantCity|MerchantState|Zip|MCC|IsFraud|Time_hour|
+----+----+----+-----+---+------+-------+------------+-------------+---+---+-------+---------+
|   0|   0|   0|    0|  0|     0|      0|           0|            0|  0|  0|      0|        0|
+----+----+----+-----+---+------+-------+------------+-------------+---+---+-------+---------+



При помощи оконных функций для каждого клиента рассчитайте средний размер транзакции, количество транзакций и последнюю по дате транзакцию.

In [30]:
trans_data.withColumn('avg_amount', F.mean('Amount').over(Window.partitionBy('User')))\
          .withColumn('cnt_trans', F.count('*').over(Window.partitionBy('User')))\
          .withColumn('last_trans_amount', F.last('Amount').over(Window.partitionBy('User').orderBy(F.col('Year').asc(),
                                                                                                     F.col('Month').asc(),
                                                                                                     F.col('Day').asc(),
                                                                                                     F.col('Time_hour').asc())))\
          .withColumn('rn_user', F.row_number().over(Window.partitionBy('User').orderBy(F.col('Year').asc(),  
                                                                                        F.col('Month').asc(),
                                                                                        F.col('Day').asc(),
                                                                                        F.col('Time_hour').asc())))\
          .withColumn('max_rn_user', F.max('rn_user').over(Window.partitionBy('User')))\
          .filter(F.col('rn_user')==F.col('max_rn_user'))\
          .select('User', 'avg_amount', 'cnt_trans', 'last_trans_amount')\
          .sort(F.col('User'))\
          .show(5) 

+----+------------------+---------+-----------------+
|User|        avg_amount|cnt_trans|last_trans_amount|
+----+------------------+---------+-----------------+
|   0| 81.29998946268704|    19963|            41.05|
|   1|    81.11805023611|     8919|            70.84|
|   2| 35.15968745276686|    41978|             1.91|
|   3|117.27760303403427|    10117|            16.51|
|   4|  97.0116982793052|    18542|            31.36|
+----+------------------+---------+-----------------+
only showing top 5 rows



Теперь самое время сгруппировать данные по каждому клиенту (можно использовать collect_list для сбора данных после агрегации)
Когда будете делать агрегацию, то возьмите только часть выборки, например, через sample, для всей выборки либо не хватит памяти, либо очень долго считать

In [40]:
frac = {u.User : (1 if u.User%10==0 else 0) for u in trans_data.select(F.col('User')).distinct().collect()}

In [60]:
trans_data_sample = trans_data.sampleBy("User", fractions=frac, seed=17)

In [44]:
trans_data_sample.count(), trans_data.count()

(2426244, 24386900)

In [45]:
# перенес всю историю по 10% клиентов
trans_data_sample.select(F.col('User')).distinct().count()

200

In [61]:
trans_data_sample = trans_data_sample.select('User', 'Card', F.struct('Amount', 'Year', 'Month', 'Day', 'Time_hour', 'MCC').alias('sequence'))\
                                     .groupBy('User', 'Card')\
                                     .agg(F.collect_list('sequence').alias('sequence'))\
                                
                                    
trans_data_sample.show(5)        

+----+----+--------------------+
|User|Card|            sequence|
+----+----+--------------------+
|   0|   1|[{56.29, 2014, 4,...|
|   0|   4|[{8.61, 2008, 9, ...|
|  40|   0|[{423.45, 1998, 3...|
|  40|   1|[{427.91, 2004, 1...|
|  40|   3|[{24.21, 2020, 1,...|
+----+----+--------------------+
only showing top 5 rows



In [63]:
trans_data_sample.take(1)[0]['sequence']

[Row(Amount=56.290000915527344, Year=2014, Month=4, Day=9, Time_hour=13, MCC=5719),
 Row(Amount=41.66999816894531, Year=2014, Month=4, Day=10, Time_hour=6, MCC=5942),
 Row(Amount=42.220001220703125, Year=2014, Month=4, Day=11, Time_hour=9, MCC=7538),
 Row(Amount=143.5500030517578, Year=2014, Month=4, Day=14, Time_hour=6, MCC=5311),
 Row(Amount=19.969999313354492, Year=2014, Month=4, Day=16, Time_hour=6, MCC=5411),
 Row(Amount=24.780000686645508, Year=2014, Month=4, Day=16, Time_hour=10, MCC=7230),
 Row(Amount=114.02999877929688, Year=2014, Month=4, Day=17, Time_hour=6, MCC=5541),
 Row(Amount=131.97999572753906, Year=2014, Month=4, Day=18, Time_hour=6, MCC=5311),
 Row(Amount=43.38999938964844, Year=2014, Month=4, Day=22, Time_hour=6, MCC=5942),
 Row(Amount=6.21999979019165, Year=2014, Month=4, Day=24, Time_hour=17, MCC=5815),
 Row(Amount=50.20000076293945, Year=2014, Month=4, Day=25, Time_hour=20, MCC=4900),
 Row(Amount=54.2599983215332, Year=2014, Month=4, Day=27, Time_hour=9, MCC=7538

Напишите python функцию, которая возьмет данные после агрегации последовательностей, отсортирует их внутри по дате и времени и преобразует к формату python dict:
{'User': User,
'Card': Card,
'sequence':{
    'amount': [последовательность],
    'year': [последовательность],
    'month': [последовательность],
    'day': [последовательность],
    'time': [последовательность],
    'MCC': [последовательность]
}
}




In [79]:
def convert_to_dict(x):

  seq_dict = {'amount': [],
              'year': [],
              'month': [],
              'day': [],
              'time': [],
              'MCC': []}
  
  seq = sorted(x['sequence'], key=lambda x: (x['Year'], x['Month'], x['Day'], x['Time_hour']))

  for row in seq:
    seq_dict['amount'].append(row['Amount'])
    seq_dict['year'].append(row['Year'])
    seq_dict['month'].append(row['Month'])
    seq_dict['day'].append(row['Day'])
    seq_dict['time'].append(row['Time_hour'])
    seq_dict['MCC'].append(row['MCC'])

  return {'User': x['User'], 'Card': x['Card'], 'sequence': seq_dict}

In [85]:
result_trans_dict = trans_data_sample.rdd.map(convert_to_dict)

Выведите как пример одну преобразованную запись, результаты сохраните на диск в через rdd pickle

In [89]:
result_trans_dict.saveAsPickleFile('trans_res')

In [87]:
result_trans_dict.take(1)[0].keys()

dict_keys(['User', 'Card', 'sequence'])

In [88]:
result_trans_dict.take(1)

[{'User': 0,
  'Card': 1,
  'sequence': {'amount': [56.290000915527344,
    41.66999816894531,
    42.220001220703125,
    143.5500030517578,
    19.969999313354492,
    24.780000686645508,
    114.02999877929688,
    131.97999572753906,
    43.38999938964844,
    6.21999979019165,
    50.20000076293945,
    54.2599983215332,
    20.610000610351562,
    2.5899999141693115,
    75.11000061035156,
    42.79999923706055,
    13.489999771118164,
    158.13999938964844,
    6.289999961853027,
    35.15999984741211,
    116.62999725341797,
    111.8499984741211,
    10.970000267028809,
    149.22000122070312,
    21.729999542236328,
    4.46999979019165,
    56.880001068115234,
    61.0,
    -61.0,
    172.2100067138672,
    42.84000015258789,
    -58.0,
    58.0,
    194.83999633789062,
    38.650001525878906,
    36.20000076293945,
    131.38999938964844,
    15.619999885559082,
    47.15999984741211,
    79.0999984741211,
    117.45999908447266,
    31.889999389648438,
    19.879999160766